
error models
https://pavpanchekha.com/blog/epsilon-delta.html
Eva https://malyzajko.github.io/


Mini herbie using z3 egraph
Direct lambdify to numpy

numpy based axiom validation.



derive bounds using z3 optimize or a branching procedure. We don't need the _tightest_ bounds.

In [23]:
import knuckledragger as kd
import knuckledragger.theories.Real as R

import numpy as np

#rnd = kd.smt.Function("rnd", kd.R, kd.R, kd.R) #fixed point rounding
#kd.axiom(kd.smt.ForAll([x, y], R.abs(rnd(x, y) - x) <= y)) # This is the important axiom to derive from

def Fixed(eps):
    F = kd.Record(f"Fixed_{eps}", ("val", kd.R ))
    x,y,z = kd.smt.Consts("x y z", F)
    a,b,c = kd.smt.Consts("a b c", kd.R)
    rnd = kd.smt.Function(f"rnd_{eps}", kd.R, F)
    #F.wf = rnd(x).val == x.val # hmm.
    F.rnd = rnd
    # F.rnd = kd.define("rnd", [x], F.mk(R.floor(x / eps) * eps))
    F.rnd_defn = kd.axiom(kd.smt.ForAll([a], R.abs(rnd(a).val - a) <= eps))
    # F.rnd_bnd = kd.axiom(kd.smt.ForAll([x, y], smt.And(eps <= rnd(x).val - x,  rnd(x).val - x.val <= eps))
    # probably not as important
    #F.rnd_idem = kd.axiom(kd.smt.ForAll([x, y], rnd(rnd(x).val).val == x))
    F.add = kd.notation.add.define([x,y], rnd(x.val + y.val))
    kd.notation.mul.define([x,y], rnd(x.val * y.val))
    kd.notation.sub.define([x,y], rnd(x.val - y.val))
    kd.notation.div.define([x,y], rnd(x.val / y.val))
    F.fma = kd.define("fma", [x,y,z], rnd(x.val * y.val + z.val))
    return F
    

F = Fixed(0.1)


a,b,c = kd.smt.Consts("a b c", F)

def mycalc(a,b,c):
    return a + b + c


kd.lemma(mycalc(a,b,c).val - mycalc(a.val, b.val, c.val) <= 0.2, by=[F.rnd_defn, F.add.defn, R.abs.defn])
# How to atudoderive 0.2?
# just use gappa or similar?
# traverse expression to find intermiedtae bounds.
def auto_bnd(x, guess=None):
    bnd = 2**10
    trykd.lemma(x <= bnd,  by=[F.rnd_defn, F.add.defn, R.abs.defn])





|- val(add(add(a, b), c)) - (val(a) + val(b) + val(c)) <= 1/5

In [ ]:

expF = kd.define("expF", [x], kd.exp(x.val))


# JUST work with lagrange remainder form
exp_approx = kd.axiom(exp(x) - (1 + x + x**2 / 2 + x**3 / 3) <= exp(x) * x**4 / 4)

# can I iterate this to arbitrary accuracy?
exp_approx = kd.axiom(exp(x) <= 1 + exp(x) * x) # lagrange form uppper boudn.
exp_lower = kd.axiom(exp(x) >= 1 + x) # lagrange form f' = exp(x) >= 1 is x >= 0. Turns out the always be true via

exp = kd.axiom(exp(x) == 1 + exp(eps) * x), 0 <= eps <= x
exp = kd.axiom(exp(x) == 1 + (1 + exp(eps2) * eps) * x ),  0 <= eps2 <= eps <= x

# we need to find the remainder point. Or have an pproximation scheme to it.
eps ~ (exp(x) - 1 - x) / (x  * exp(eps2)) 

# some kind of guess of eps(x) would help.
exp(x) >= 0
exp(eps) >= 1 # becasue eps >= 0
1 + x + x**2 +  = 1 / 1 - x

# exp(x) <= (1 + x) / (1 + x**2/2)? Huh 

# finite integration
exp(x) = exp(x/2) + exp(eps) (x  - x/2)
exp(x/2) = exp(0) + exp(eps1) (x/2 - 0)
exp(x) = exp(0) + exp(eps1) (x/2) + exp(eps) (x/2) , 0 <= eps1 <= x/2 <= eps <= x

# tube

pow2(x) <= exp(x) <= pow2(x+1) # 2^x <= e^x <= 4^x aka 2 <= e <= 4

And(Implies( box(x,0,1), box(exp(x), 1, )))


# we're asying it obeys an integral eqaution. But we havve to use a fixed point lemma like picard lindelof to show there is value
# we've definied it but we need another principle to bootstrap an actual value.

Implies( QForAll([x], 0 <= x <= 1, b <= exp(x) <= a),  d < exp(x) < c  )  


(i1 -> i2) -> (i2 -> i3)


bisect_f(I) = If( mid(i) <= exp())

def box(l,x,u):
    return And(l <= x, x <= u)

sqrt(x) = sqrt(1) + 

ln(x) = ln(1) + 1 / eps * (x - 1), 1 <= eps <= x

ln(1 - x) = ln(1 - x/2) + 1/ eps * (x/2)
ln(1-x/2) = ln(1) + 1/eps1 * (x/2 - 1)

eps * ln(1-x) = x  # exists 1 <= eps <= x st 
# 1 <= ln() <= x 

ln(1 - x) = 

# https://en.wikipedia.org/wiki/Inverse_function_theorem + mvt
exp(x) = ln^-1(x) = exp(0) + ( 1 / exp(x) ) x # ???


bisect_sqrt(x,a,b) = If((a + b / 2)**2 <= x, bisect_sqrt(x, a, (a + b) / 2), bisect_sqrt(x, (a + b) / 2, b))

# watch out and propagate going out of bounds
def Fixed2(eps, max):
    F = Datatype(f"Fixed_{eps}_{max}")
    F.declare("NaN")
    F.declare("Inf")

https://en.wikipedia.org/wiki/Banach_fixed-point_theorem
Basically, geometric series bound on norm


geom(x,n) = x**n




# FPTalks


https://colab.research.google.com/drive/1bqXS207E4IerbiXSTGKm1KIKAHyjCK50

In [2]:
import numpy as np
def ulp(func=np.float32):
    u = func(1)
    while func(1)+func(u) != func(1):
        u_last = u
        u = func(u) / func(2)
    return u_last
ulp32=ulp(np.float32)
ulp32

1.1920929e-07

eva  - compiler for the reals

nominal completion nd slotted egraphs
f(e2(a), e3(a,b)) -> e1(a,b)

op = 1+e) + d
d for denormla

posits

x*x + y*y

datflow - fatser
daisy
fluctuat

optimzation - optimization
fptaylor
precisa

non modular, no loops
SAS modulao

hugo tool.

<https://github.com/soarlab/FPTaylor>
<https://github.com/malyzajko/daisy>

(Handbook of mathemtical functions)
Kalmand filters

heat equation loop

satire tool and fluctat
<https://github.com/arnabd88/Satire>

<https://github.com/soarlab/gelpia>

conditionals. are still tough

a and b are *atoms*
So we are 3 flat instead of 2 flat

Propel talk diseaqlity forbid. Good chat with him.

https://github.com/metalibm/metalibm

https://gappa.gitlabpages.inria.fr/gappa/index.html

In [2]:
%%file /tmp/testgappa.g
{ x in [-2,2] -> x * x in ? }

Writing /tmp/testgappa.g


In [6]:
! gappa -Blatex /tmp/testgappa.g | pandoc -f latex -t markdown

Results:
  x * x in [0, 4]
Under the following hypotheses $$\label{g1}
\left( \left| \mathit{x} \right| \le2 \land \neg\left( \mathit{x} \times \mathit{x} \in [0,4] \right) \right),$$
one can deduce the following properties: $$\label{l2}
\neg\left( \mathit{x} \times \mathit{x} \in [0,4] \right)$$ by using
([\[g1\]](#g1){reference-type="ref" reference="g1"}), and selecting a
component. $$\label{l5}
\left| \mathit{x} \right| \le2$$ by using
([\[g1\]](#g1){reference-type="ref" reference="g1"}), and selecting a
component. $$\label{l4}
\left| \mathit{x} \right| \le 2$$ by using
([\[l5\]](#l5){reference-type="ref" reference="l5"}), and theorem
`abs_of_bnd_o`. $$\label{l3}
\mathit{x} \times \mathit{x} \in [0,4]$$ by using
([\[l4\]](#l4){reference-type="ref" reference="l4"}), and theorem
`square`. $$\label{l1}
\bot$$ by using ([\[l2\]](#l2){reference-type="ref" reference="l2"}),
([\[l3\]](#l3){reference-type="ref" reference="l3"}), and discarding
contradictory literals.


In [ ]:
import subprocess
def gappa(f):
    


In [ ]:
ex1 = """{ x in [0,1] -> float<ieee_32,ne>(x * float<ieee_32,ne>(1 - x)) in ? }"""
gappa(ex1)
